In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.tensorboard.writer import SummaryWriter

from sklearn.model_selection import train_test_split

from utils.data import get_dataset_paths
from utils.data import Marconi100Dataset
from utils.data import UnfoldedDataset
from utils.training import training_loop

from algos.deep_fib.core import get_masks
from algos.deep_fib.core import DeepFIBEngine
from algos.deep_fib.sci_net import SCINet


In [3]:
paths = get_dataset_paths("../data")
train, test = train_test_split(paths, test_size=0.1, random_state=42)


In [4]:
m_data_train = Marconi100Dataset(train, scaling="minmax")
m_data_test = Marconi100Dataset(test, scaling="minmax")


Loading:  57%|█████▋    | 128/224 [04:53<04:14,  2.65s/it]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

horizon = 1024
stride = 512
n_masks = 100

batch_size = 32

num_encoder_levels = 2

log_dir = "./outputs/deep_fib"
lr = 1e-4
num_epochs = 10

hidden = None
input_dim = 460
hidden_size = 4
kernel_size = 3
dropout = 0.5

anomaly_threshold = 0.1  # to be tuned


In [ ]:
dataset_train = UnfoldedDataset(m_data_train, horizon=horizon, stride=stride)
dataset_test = UnfoldedDataset(m_data_test, horizon=horizon, stride=stride)

masks = get_masks(horizon, n_masks).float()

len(dataset_train), len(dataset_test), masks.size()


In [ ]:
train_loader = DataLoader(
    dataset_train,
    batch_size,
    shuffle=True,
)
test_loader = DataLoader(
    dataset_test,
    batch_size,
    shuffle=False,
)
len(train_loader), len(test_loader)


In [ ]:
model = SCINet(
    output_len=horizon,
    input_len=horizon,
    num_encoder_levels=num_encoder_levels,
    hidden_decoder_sizes=hidden,
    input_dim=input_dim,
    hidden_size=hidden_size,
    kernel_size=kernel_size,
    dropout=dropout,
).float()

engine = DeepFIBEngine(anomaly_threshold, masks)

optim = Adam(model.parameters(), lr=lr)
lr_sched = CosineAnnealingLR(optim, num_epochs)


In [ ]:
with SummaryWriter(log_dir) as writer:
    training_loop(
        model=model,
        engine=engine,
        num_epochs=num_epochs,
        train_dataloader=train_loader,
        test_dataloader=test_loader,
        device=device,
        optimizer=optim,
        lr_scheduler=lr_sched,
        writer=writer,
        save_path=log_dir + "/models",
    )


# Test

In [1]:
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.tensorboard.writer import SummaryWriter
import numpy as np
from sklearn.model_selection import train_test_split

from utils.data import get_dataset_paths
from utils.data import Marconi100Dataset
from utils.data import UnfoldedDataset
from utils.modutils import load_model

from algos.deep_fib.core import get_masks
from algos.deep_fib.core import DeepFIBEngine
from algos.deep_fib.sci_net import SCINet

In [2]:
paths = get_dataset_paths("../data")
train, test = train_test_split(paths, test_size=0.01, random_state=42)

m_data_test = Marconi100Dataset(test, scaling="minmax")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

Loading: 100%|██████████| 3/3 [00:06<00:00,  2.10s/it]

cuda


In [3]:
data = m_data_test[0]

In [4]:
horizon = 1024
stride = 512
n_masks = 100

batch_size = 32

num_encoder_levels = 2

log_dir = "./outputs/deep_fib"
lr = 1e-2
num_epochs = 20

hidden = None
input_dim = 460
hidden_size = 4
kernel_size = 3
dropout = 0.5

In [5]:
dataset_test = UnfoldedDataset(m_data_test, horizon=horizon, stride=stride)

print(len(dataset_test))

test_loader = DataLoader(
    dataset_test,
    batch_size,
    shuffle=False,
)
print(len(test_loader))

79
3


In [10]:
model = SCINet(
    output_len=horizon,
    input_len=horizon,
    num_encoder_levels=num_encoder_levels,
    hidden_decoder_sizes=hidden,
    input_dim=input_dim,
    hidden_size=hidden_size,
    kernel_size=kernel_size,
    dropout=dropout,
).float()

load_model(model, "outputs/deep_fib/models/model_8.pth", device)

SCINet(
  (encoder): EncoderTree(
    (workingblock): SCINetBlock(
      (rho): Interactor(
        (0): ReplicationPad1d((2, 2))
        (1): Conv1d(460, 1840, kernel_size=(3,), stride=(1,))
        (2): LeakyReLU(negative_slope=0.01, inplace=True)
        (3): Dropout(p=0.5, inplace=False)
        (4): Conv1d(1840, 460, kernel_size=(3,), stride=(1,))
        (5): Tanh()
      )
      (eta): Interactor(
        (0): ReplicationPad1d((2, 2))
        (1): Conv1d(460, 1840, kernel_size=(3,), stride=(1,))
        (2): LeakyReLU(negative_slope=0.01, inplace=True)
        (3): Dropout(p=0.5, inplace=False)
        (4): Conv1d(1840, 460, kernel_size=(3,), stride=(1,))
        (5): Tanh()
      )
      (phi): Interactor(
        (0): ReplicationPad1d((2, 2))
        (1): Conv1d(460, 1840, kernel_size=(3,), stride=(1,))
        (2): LeakyReLU(negative_slope=0.01, inplace=True)
        (3): Dropout(p=0.5, inplace=False)
        (4): Conv1d(1840, 460, kernel_size=(3,), stride=(1,))
        (5): 

In [29]:
13000 * 250

3250000

In [30]:
26000 / (13000 * 250)

0.008

In [11]:
anomaly_threshold = 0.4  # to be tuned
engine = DeepFIBEngine(anomaly_threshold)

for batch in test_loader:
    batch = {k: d.to(device) for k, d in batch.items()}
    rets = engine.test_step(model, batch)
    break

rets

{'errors': tensor([[0.1328, 0.1284, 0.1203,  ..., 0.3008, 0.2402, 0.2255],
         [0.1007, 0.0993, 0.0997,  ..., 0.2704, 0.2553, 0.2700],
         [0.2143, 0.2036, 0.1961,  ..., 0.2331, 0.2530, 0.2530],
         ...,
         [0.1539, 0.1584, 0.1560,  ..., 0.2044, 0.1963, 0.1981],
         [0.1372, 0.1381, 0.1364,  ..., 0.3513, 0.3434, 0.3514],
         [0.1972, 0.2041, 0.2036,  ..., 0.2338, 0.2322, 0.2327]],
        device='cuda:0'),
 'labels': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0', dtype=torch.int32)}

In [18]:
rets["errors"].mean()

tensor(0.2239, device='cuda:0')

In [24]:
anomaly_threshold = 0.1  # to be tuned
engine = DeepFIBEngine(anomaly_threshold)

from collections import defaultdict

test_scalars = defaultdict(list)
for batch in test_loader:
    batch = {k: d.to(device) for k, d in batch.items()}
    rets = engine.val_step(model, batch)
    for tag, val in rets.items():
        test_scalars[tag].append(float(val))

for tag, val in test_scalars.items():
    print(tag, np.mean(val))

f1 0.037855809554457664
precision 0.01937031528602044
recall 0.9145428737004598
loss 0.24163837730884552


In [3]:
horizon = 1024
stride = 512
n_masks = 100

batch_size = 32

num_encoder_levels = 3

lr = 1e-2
num_epochs = 20

hidden = [512]
input_dim = 460
hidden_size = 2
kernel_size = 3
dropout = 0.5

In [4]:
model = SCINet(
    output_len=horizon,
    input_len=horizon,
    num_encoder_levels=num_encoder_levels,
    hidden_decoder_sizes=hidden,
    input_dim=input_dim,
    hidden_size=hidden_size,
    kernel_size=kernel_size,
    dropout=dropout,
).float()

In [5]:
masks = get_masks(horizon, 10).float()

In [6]:
masks.size()

torch.Size([10, 1024, 460])

In [7]:
dataset_test = UnfoldedDataset(m_data_test, horizon=horizon, stride=stride)

print(len(dataset_test))

test_loader = DataLoader(
    dataset_test,
    batch_size,
    shuffle=False,
)
print(len(test_loader))

79
3


In [8]:
import gc

model.to(device)
anomaly_threshold = 0.4  # to be tuned
engine = DeepFIBEngine(anomaly_threshold, masks)

for batch in test_loader:
    batch = {k: d.to(device) for k, d in batch.items()}
    rets = engine.train_step(model, batch)
    del rets
    gc.collect()